# LSTM

## 数据处理

In [15]:
# encoding=utf8
import io
import os
import re
import sys
import random
import requests
from collections import OrderedDict 
import math
import numpy as np
import paddle
from paddle.nn import Embedding
import paddle.nn.functional as F
import tarfile
from paddle.nn import LSTM, Embedding, Dropout, Linear
import paddlenlp

In [16]:
#应用库内数据，下载速度慢，所以直接用url下到工作区
#train_ds, test_ds = paddlenlp.datasets.load_dataset("ptb", splits=("train", "test"))

"""
def download():
    # 通过python的requests类，下载存储在
    # http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
    corpus_url = "http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz"
    web_request = requests.get(corpus_url)
    corpus = web_request.content

    # 将下载的文件写在当前目录的aclImdb_v1.tar.gz文件内
    with open("./aclImdb_v1.tar.gz", "wb") as f:
        f.write(corpus)
    f.close()

download()"""
#也慢，所以在本地下载数据后，挂载到云上
#因为此前的代码根据API中的数据格式进行处理，和本地的数据格式不一样，在此处构建loadtext转变格式
def loadtxt(path):
    file = open(path)
    lines=[]
    for line in file.readlines():
        line = line.strip(" \n ")
        lines.append(line)
    data=[]
    for sentence in lines:
        one_data={}
        one_data['sentence']=sentence
        data.append(one_data)
    return data

train_path='ptb.train.txt'
test_path='ptb.test.txt'

train_ds = loadtxt(train_path)
test_ds =loadtxt(test_path)

数据集信息一览

In [17]:
print(np.array(train_ds[0]))
print(train_ds[0]['sentence']) #字典
print(len(train_ds))
print(len(test_ds))

"""for i in range(len(test_ds)):
    print(len(train_ds[i]['sentence']))"""
#句长不等

{'sentence': 'aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter'}
aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter
42068
3761


"for i in range(len(test_ds)):\n    print(len(train_ds[i]['sentence']))"

In [18]:
sentence=train_ds[0]['sentence']
#sentence.strip().lower()
sentence.split(" ")

['aer',
 'banknote',
 'berlitz',
 'calloway',
 'centrust',
 'cluett',
 'fromstein',
 'gitano',
 'guterman',
 'hydro-quebec',
 'ipo',
 'kia',
 'memotec',
 'mlx',
 'nahb',
 'punts',
 'rake',
 'regatta',
 'rubens',
 'sim',
 'snack-food',
 'ssangyong',
 'swapo',
 'wachter']

In [19]:
def data_preprocess(dataset):
    data_set = []
    for i in range(len(dataset)):
        # 这里有一个小trick是把所有的句子转换为小写，从而减小词表的大小
        # 一般来说这样的做法有助于效果提升
        sentence=train_ds[i]['sentence']
        sentence = sentence.strip().lower()
        sentence = sentence.split(" ")
        
        data_set.append(sentence)
    return data_set

train_corpus = data_preprocess(train_ds)
test_corpus = data_preprocess(test_ds)
print(train_corpus[:5])
print(test_corpus[:5])

[['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter'], ['pierre', '<unk>', 'n', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'n'], ['mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group'], ['rudolph', '<unk>', 'n', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate'], ['a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of', 'workers', 'exposed', 'to', 'it', 'more', 'than', 'n', 'years', 'ago', 'researchers', 'reported']]
[['aer', 'b

In [20]:
# 构造词典，统计每个词的频率，并根据频率将每个词转换为一个整数id
def build_dict(corpus):
    word_freq_dict = dict()
    for sentence in corpus:
        for word in sentence:
            if word not in word_freq_dict:
                word_freq_dict[word] = 0
            word_freq_dict[word] += 1

    word_freq_dict = sorted(word_freq_dict.items(), key = lambda x:x[1], reverse = True)
    
    word2id_dict = dict()
    word2id_freq = dict()

    # 一般来说，我们把unk和pad放在词典前面，给他们一个比较小的id，这样比较方便记忆，并且易于后续扩展词表
    word2id_dict['<unk>'] = 0
    word2id_freq[0] = 1e10

    word2id_dict['[pad]'] = 1
    word2id_freq[1] = 1e10

    for word, freq in word_freq_dict:
        word2id_dict[word] = len(word2id_dict)
        word2id_freq[word2id_dict[word]] = freq

    return word2id_freq, word2id_dict

word2id_freq, word2id_dict = build_dict(train_corpus)
vocab_size = len(word2id_freq)
print("there are totoally %d different words in the corpus" % vocab_size)
for _, (word, word_id) in zip(range(10), word2id_dict.items()):
    print("word %s, its id %d, its word freq %d" % (word, word_id, word2id_freq[word_id]))

there are totoally 10000 different words in the corpus
word <unk>, its id 3, its word freq 32481
word [pad], its id 1, its word freq 10000000000
word the, its id 2, its word freq 50770
word n, its id 3, its word freq 32481
word of, its id 4, its word freq 24400
word to, its id 5, its word freq 23638
word a, its id 6, its word freq 21196
word in, its id 7, its word freq 18000
word and, its id 8, its word freq 17474
word 's, its id 9, its word freq 9784


In [21]:
# 把语料转换为id序列
def convert_corpus_to_id(corpus, word2id_dict):
    data_set = []
    for sentence in corpus:
        # 将句子中的词逐个替换成id，如果句子中的词不在词表内，则替换成'<unk>'
        # 这里需要注意，一般来说我们可能需要查看一下test-set中，句子'<unk>'的比例，
        # 如果存在过多'<unk>'的情况，那就说明我们的训练数据不足或者切分存在巨大偏差，需要调整
        sentence = [word2id_dict[word] if word in word2id_dict \
                    else word2id_dict['<unk>'] for word in sentence]    
        data_set.append(sentence)
    return data_set

train_corpus = convert_corpus_to_id(train_corpus, word2id_dict)
test_corpus = convert_corpus_to_id(test_corpus, word2id_dict)
print("%d tokens in the corpus" % len(train_corpus))
print(train_corpus[:5])
print(test_corpus[:5])

42068 tokens in the corpus
[[9970, 9971, 9972, 9973, 9974, 9975, 9976, 9977, 9978, 9979, 9980, 9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9989, 9990, 9991, 9992, 9993], [8569, 3, 3, 72, 393, 33, 2116, 2, 146, 19, 6, 8570, 275, 407, 3], [23, 3, 13, 141, 4, 3, 5278, 2, 3055, 1581, 96], [7232, 3, 3, 72, 393, 8, 337, 141, 4, 2468, 657, 2158, 949, 24, 521, 6, 8570, 275, 4, 39, 303, 437, 3661], [6, 941, 4, 3143, 495, 262, 5, 137, 5882, 4219, 5883, 30, 986, 6, 240, 755, 4, 1013, 2765, 211, 6, 96, 4, 427, 4060, 5, 14, 45, 55, 3, 72, 195, 1233, 220]]
[[9970, 9971, 9972, 9973, 9974, 9975, 9976, 9977, 9978, 9979, 9980, 9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9989, 9990, 9991, 9992, 9993], [8569, 3, 3, 72, 393, 33, 2116, 2, 146, 19, 6, 8570, 275, 407, 3], [23, 3, 13, 141, 4, 3, 5278, 2, 3055, 1581, 96], [7232, 3, 3, 72, 393, 8, 337, 141, 4, 2468, 657, 2158, 949, 24, 521, 6, 8570, 275, 4, 39, 303, 437, 3661], [6, 941, 4, 3143, 495, 262, 5, 137, 5882, 4219, 5883, 30, 986, 6, 240, 755, 4

在PTB数据集中，每个句子并非随机抽取的文本，而是在上下文之间有关联的内容。语言模型为了利用上下文信息，必须将前面句子的信息传递到后面的句子。为了实现这个目标，在PTB上下文有关联的数据集上，通常采用另一种batching方法。

将长序列切割为固定长度的子序列。循环神经网络在处理完一个子序列后，它最终的隐藏状态将复制到下一个序列中作为初始值，这样在前向计算时，效果等同于一次性顺序地读取了整个文档；而在反向传播时，梯度则只在每个子序列内部传播。

为了利用计算时的并行能力，希望每一次计算可以对多个句子进行并行处理，同时又要尽量保证batch之间的上下文连续。解决方案是，先将整个文档分成若干连续段落，再让batch中的每一个位置负责其中一段。例如，如果batch大小时4，则先将这个文档平均分成4个子序列，让batch中的每一个位置负责其中一个子序列，这样每个子文档内部的所有数据仍可以被顺序处理。

下面的代码从文件中读取数据，并按上面介绍的方案将数据整理成batch。由于PTB数据集比较小，因此可以直接将这个数据集一次性读入内存。

In [22]:
TRAIN_BATCH_SIZE = 20               # 训练数据batch的大小
TRAIN_NUM_STEP = 35                 # 训练数据截断长度

#生成保存上下文信息的总文本id_list，用它来构造训练batch
def make_id_list(corpus):
    id_list=[]
    for sentence in corpus:
        for word in sentence:
            id_list.append(word)
    return id_list

#train_id_list=make_id_list(test_corpus)
#print(train_id_list[:50])

def make_batch(id_list, batch_size, num_step):
    # 计算总的batch数量，每个batch包含的单词数量是batch_size * num_step
    num_batches = (len(id_list) - 1) // (batch_size * num_step)
    # 将数据整理成一个维度为[batch_size, num_batches * num_step]的二维数组
    data = np.array(id_list[: num_batches * batch_size * num_step])
    data = np.reshape(data, [batch_size, num_batches * num_step])
    # 沿着第二个维度将数据切分成num_batches个batch,存入一个数组
    data_batches = np.split(data, num_batches, axis=1)

    # 重复上述操作，但是每个位置向右移动一位，这里得到的是RNN每一步输出所需要预测的下一个单词
    label = np.array(id_list[1: num_batches * batch_size * num_step + 1])
    label = np.reshape(label, [batch_size, num_batches * num_step])
    label_batches = np.split(label, num_batches, axis=1)
    # 返回一个长度为num_batches的数组，其中每一项包含一个data矩阵和一个label矩阵
    return list(zip(data_batches, label_batches))


train_id_list=make_id_list(train_corpus)
train_batch=make_batch(train_id_list,TRAIN_BATCH_SIZE,TRAIN_NUM_STEP)

In [23]:
print(len(train_batch))#113个batch

print(len(train_batch[0]))#每一个batch有2个矩阵：数据和标签

print(len(train_batch[0][0]))#数据矩阵有20个句子，标签矩阵中有20个词矩阵

print(len(train_batch[0][0][0]))#一个句子中有35个词

print(train_batch[0][0][0])

1267
2
20
35
[9970 9971 9972 9973 9974 9975 9976 9977 9978 9979 9980 9981 9982 9983
 9984 9985 9986 9987 9988 9989 9990 9991 9992 9993 8569    3    3   72
  393   33 2116    2  146   19    6]


In [24]:
print(train_batch[0][0][0])#第一个batch中的第一个句子

print(train_batch[0][1][0])#第一个batch中输出预测的标签矩阵

print(train_batch[0][0][1])

print(train_batch[0][1][1])

#标签矩阵的最后一个值，就是对应输入句子的真实next word

[9970 9971 9972 9973 9974 9975 9976 9977 9978 9979 9980 9981 9982 9983
 9984 9985 9986 9987 9988 9989 9990 9991 9992 9993 8569    3    3   72
  393   33 2116    2  146   19    6]
[9971 9972 9973 9974 9975 9976 9977 9978 9979 9980 9981 9982 9983 9984
 9985 9986 9987 9988 9989 9990 9991 9992 9993 8569    3    3   72  393
   33 2116    2  146   19    6 8570]
[1569  106  138  767   14   13 1513   18 1446    2  844  236    2 1384
    5 1273    7 1635 1089 3842   17  380 1352    4  207    2 2600    4
    2  261   13    5  335    2    3]
[ 106  138  767   14   13 1513   18 1446    2  844  236    2 1384    5
 1273    7 1635 1089 3842   17  380 1352    4  207    2 2600    4    2
  261   13    5  335    2    3   16]


## 模型构建

In [25]:

class WordPredictor(paddle.nn.Layer):
    
    def __init__(self, hidden_size, vocab_size, embedding_size, prediction_num=10000, num_steps=35, num_layers=1, init_scale=0.1, dropout_rate=None):
        
        # 参数含义如下：
        # 1.hidden_size，表示embedding-size，hidden和cell向量的维度
        # 2.vocab_size，模型可以考虑的词表大小
        # 3.embedding_size，表示词向量的维度
        # 4.prediction_num,预测输出词的数量，每次预测输出对10000个词的概率
        # 5.num_steps，表示这个情感分析模型最大可以考虑的句子长度
        # 6.num_layers，表示网络的层数
        # 7.dropout_rate，表示使用dropout过程中失活的神经元比例
        # 8.init_scale，表示网络内部的参数的初始化范围,长短时记忆网络内部用了很多Tanh，Sigmoid等激活函数，\
        # 这些函数对数值精度非常敏感，因此我们一般只使用比较小的初始化范围，以保证效果
        super(WordPredictor, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.prediction_num = prediction_num
        self.num_steps = num_steps
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate
        self.init_scale = init_scale
       
        # 声明一个LSTM模型，用来把每个句子抽象成向量
        self.simple_lstm_rnn = paddle.nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers)

        # 声明一个embedding层，用来把句子中的每个词转换为向量
        self.embedding = paddle.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, sparse=False, 
                                    weight_attr=paddle.ParamAttr(initializer=paddle.nn.initializer.Uniform(low=-init_scale, high=init_scale)))
        
        # 声明使用上述语义向量映射到具体词时所需要使用的线性层
        self.pred_fc = paddle.nn.Linear(in_features=self.hidden_size, out_features=self.prediction_num, 
                             weight_attr=None, bias_attr=None)
        
        self.embedding_out = paddle.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size, sparse=False, 
                                    weight_attr=paddle.ParamAttr(initializer=paddle.nn.initializer.Uniform(low=-init_scale, high=init_scale)))

        # 一般在获取单词的embedding后，会使用dropout层，防止过拟合，提升模型泛化能力
        self.dropout_layer = paddle.nn.Dropout(p=self.dropout_rate, mode='upscale_in_train')

    # forwad函数即为模型前向计算的函数，它有两个输入，分别为：
    # input为输入的训练文本，其shape为[batch_size, max_seq_len]
    # label训练文本对应的情感标签，其shape维[batch_size, 1]
    def forward(self, inputs):
        # 获取输入数据的batch_size
        batch_size = inputs.shape[0]

        # 本实验默认使用1层的LSTM，首先我们需要定义LSTM的初始hidden和cell，这里我们使用0来初始化这个序列的记忆
        init_hidden_data = np.zeros(
            (self.num_layers, batch_size, self.hidden_size), dtype='float32')
        init_cell_data = np.zeros(
            (self.num_layers, batch_size, self.hidden_size), dtype='float32')

        # 将这些初始记忆转换为飞桨可计算的向量，并且设置stop_gradient=True，避免这些向量被更新，从而影响训练效果
        init_hidden = paddle.to_tensor(init_hidden_data)
        init_hidden.stop_gradient = True
        init_cell = paddle.to_tensor(init_cell_data)
        init_cell.stop_gradient = True

        # 对应以上第2步，将输入的句子的mini-batch转换为词向量表示，转换后输入数据shape为[batch_size, max_seq_len, embedding_size]
        x_emb = self.embedding(inputs)
        x_emb = paddle.reshape(x_emb, shape=[-1, self.num_steps, self.embedding_size])
        # 在获取的词向量后添加dropout层
        if self.dropout_rate is not None and self.dropout_rate > 0.0:
            x_emb = self.dropout_layer(x_emb)
        
        # 对应以上第3步，使用LSTM网络，把每个句子转换为语义向量
        # 返回的last_hidden即为最后一个时间步的输出，其shape为[self.num_layers, batch_size, hidden_size]
        rnn_out, (last_hidden, last_cell) = self.simple_lstm_rnn(x_emb, (init_hidden, init_cell))
        
        # 提取最后一层隐状态作为文本的语义向量，其shape为[batch_size, hidden_size]
        last_hidden = paddle.reshape(last_hidden[-1], shape=[-1, self.hidden_size])

        # 对应以上第4步，将每个句子的向量表示映射到具体的词上, logits的维度为[batch_size, 10000]
        logits = self.pred_fc(last_hidden)
        

        return logits

## 训练和验证

In [30]:
# 定义训练参数
epoch_num = 20
batch_size = 20

learning_rate = 0.01
dropout_rate = 0.2
num_layers = 1
hidden_size = 256
embedding_size = 256
max_seq_len = 35
vocab_size = len(word2id_freq)

# 检测是否可以使用GPU，如果可以优先使用GPU
use_gpu = True if paddle.get_device().startswith("gpu") else False
if use_gpu:
    paddle.set_device('gpu:0')

# 实例化模型
word_predictor = WordPredictor(hidden_size, vocab_size, embedding_size,  num_steps=max_seq_len, num_layers=num_layers, dropout_rate=dropout_rate)

# 指定优化策略，更新模型参数
optimizer = paddle.optimizer.Adam(learning_rate=learning_rate, beta1=0.9, beta2=0.999, parameters= word_predictor.parameters()) 

# 定义训练函数
# 记录训练过程中的损失变化情况，可用于后续画图查看训练情况
losses = []
steps = []
step = 0

def train(model):
    # 开启模型训练模式
    model.train()
    step = 0
    epoch_num = 20
    # 建立训练数据生成器，每次迭代生成一个batch，每个batch包含训练文本和文本对应的预测输出
    train_id_list=make_id_list(train_corpus)
    train_loader = make_batch(train_id_list,batch_size,max_seq_len)
    
    while(epoch_num>0):
        epoch_num=epoch_num-1
        for batch_data in (train_loader):
            sentences = batch_data[0]
            labels = np.zeros([batch_size,len(word2id_freq)])
            for i in range(batch_size):
                labels[i][batch_data[1][i][max_seq_len-1]] = 1
            # 获取数据，并将张量转换为Tensor类型
            sentences = paddle.to_tensor(sentences)
            labels=labels.astype("float32")
            labels = paddle.to_tensor(labels)
        
            # 前向计算，将数据feed进模型，并得到预测的情感标签和损失
            logits = model(sentences)

            # 计算损失
            loss = F.softmax_with_cross_entropy(logits, label=labels,soft_label=True)
            loss = paddle.mean(loss)

            # 后向传播
            loss.backward()
            # 更新参数
            optimizer.step()
            # 清除梯度
            optimizer.clear_grad()
            step+=1
            if step % 100 == 0:
                # 记录当前步骤的loss变化情况
                losses.append(loss.numpy()[0])
                steps.append(step)
                # 打印当前loss数值
                print("step %d, loss %.3f" % (step, loss.numpy()[0]))

#训练模型
train(word_predictor)

# 保存模型，包含两部分：模型参数和优化器参数
model_name = "word_predictor"
# 保存训练好的模型参数
paddle.save(word_predictor.state_dict(), "{}.pdparams".format(model_name))
# 保存优化器参数，方便后续模型继续训练
paddle.save(optimizer.state_dict(), "{}.pdopt".format(model_name))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: 
API "paddle.nn.functional.loss.softmax_with_cross_entropy" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.nn.functional.cross_entropy" instead.
reason: Please notice that behavior of "paddle.nn.functional.softmax_with_cross_entropy" and "paddle.nn.functional.cross_entropy" is different. 


step 100, loss 7.937
step 200, loss 7.471
step 300, loss 7.219
step 400, loss 6.024
step 500, loss 5.731
step 600, loss 6.277
step 700, loss 6.498
step 800, loss 6.513
step 900, loss 6.056
step 1000, loss 6.101
step 1100, loss 8.410
step 1200, loss 6.955
step 1300, loss 6.290
step 1400, loss 5.890
step 1500, loss 6.334
step 1600, loss 6.209
step 1700, loss 4.862
step 1800, loss 5.621
step 1900, loss 6.028
step 2000, loss 5.741
step 2100, loss 5.978
step 2200, loss 5.775
step 2300, loss 5.538
step 2400, loss 5.722
step 2500, loss 6.435
step 2600, loss 5.310
step 2700, loss 5.776
step 2800, loss 6.395
step 2900, loss 6.455
step 3000, loss 6.220
step 3100, loss 5.336
step 3200, loss 5.688
step 3300, loss 6.471
step 3400, loss 5.965
step 3500, loss 6.319
step 3600, loss 5.569
step 3700, loss 5.505
step 3800, loss 4.938
step 3900, loss 4.927
step 4000, loss 5.030
step 4100, loss 4.996
step 4200, loss 5.127
step 4300, loss 5.208
step 4400, loss 4.576
step 4500, loss 4.822
step 4600, loss 4.9

In [51]:
def test(model):
    # 开启模型测试模式，在该模式下，网络不会进行梯度更新
    model.eval()
    batch_size=20
    max_seq_len=35
    # 建立训练数据生成器，每次迭代生成一个batch，每个batch包含训练文本和文本对应的预测输出
    test_id_list=make_id_list(test_corpus)
    test_loader = make_batch(test_id_list,batch_size,max_seq_len)
    for batch_data in (test_loader):
        sentences = batch_data[0]
        labels = np.zeros([batch_size,len(word2id_freq)])
        for i in range(batch_size):
            labels[i][batch_data[1][i][max_seq_len-1]] = 1
        # 获取数据，并将张量转换为Tensor类型
        sentences = paddle.to_tensor(sentences)
        labels=labels.astype("float32")
        labels = paddle.to_tensor(labels)
    
        # 前向计算，将数据feed进模型，并得到预测的词
        logits = model(sentences)

        loss = F.cross_entropy(logits, label=labels,soft_label=True)
        loss = paddle.mean(loss)
        print(loss.numpy()[0])

test(word_predictor)

2.2248187
2.0698133
2.5195918
2.6731122
2.781458
2.5506375
2.5229526
2.7695343
1.7921125
1.5301162
2.0458202
2.1600127
2.2089536
2.5916903
2.1193178
2.291356
2.5017333
1.8197747
2.2765024
2.448466
2.7207272
2.235856
2.9058704
0.9083152
3.3780913
2.2738767
3.034109
2.802174
2.070563
2.9802952
1.7074177
3.0309887
2.3154252
2.7516403
2.2980306
3.0308218
3.2759545
3.2880511
2.392037
2.1802344
2.2167194
1.8243294
2.8874636
2.8581574
2.51129
3.2798755
3.4761224
2.9465032
2.477037
3.7621474
2.1338117
2.5772882
2.4724739
3.0121753
2.6000204
2.514338
2.7832594
1.8919666
2.7685692
3.1906834
3.1233087
2.7859933
2.7595596
2.7127957
2.506654
2.4585955
2.9025521
2.6337223
2.4671838
1.5749327
2.1385121
3.3348565
2.655046
2.8515453
2.7485979
3.485397
2.5329626
2.4860525
2.958406
1.9912493
2.4883237
3.0432968
2.9127862
2.5964115
2.359677
3.1654375
2.3604064
2.2860062
3.3965492
3.0826578
2.788619
1.7307892
2.231455
2.20822
1.539428
2.6197343
2.5535545
1.8368229
2.9541156
2.929738
3.0342336
3.0388548
2.7

任务基本完成。

还需要改进的地方：

输出转化为词，计算accuracy，校订损失（有闲再搞）